In [1]:
import numpy as np
import cv2
import os
import random
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import model_from_json
from shutil import copyfile
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
import matplotlib.pyplot as plt

In [2]:
path = r"F:\Deep Learning\Facial Recognision\haarcascade_frontalface_default.xml"

In [3]:
face_cascade = cv2.CascadeClassifier(path)

In [4]:
model = load_model("my_model.h5")
model.load_weights('fer1.h5')

In [5]:
datapath = r"F:\Deep Learning\Facial Recognision\Facial Recognesion"
train_datapath = r"F:\Deep Learning\Facial Recognision\Facial Recognesion\Training"
validation_datapath = r"F:\Deep Learning\Facial Recognision\Facial Recognesion\Testing"
validation_datagen = ImageDataGenerator(rescale = 1/255)
validation_generator = validation_datagen.flow_from_directory(
    validation_datapath , 
    batch_size = 32,
    target_size = (48 , 48),
    color_mode='grayscale',
    class_mode = 'categorical'
)

Found 7067 images belonging to 6 classes.


In [6]:
from tensorflow.keras.optimizers import Adam , SGD
opt = SGD(lr = 0.0001 , momentum=0.9 , decay = 1e-4/25)
model.compile(loss = 'categorical_crossentropy' , optimizer = opt ,  
             metrics = ['accuracy'])

In [7]:
loss, acc = model.evaluate(validation_generator, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

  ...
    to  
  ['...']
221/221 - 17s - loss: 2.5427 - accuracy: 0.5749
Restored model, accuracy: 57.49%


In [8]:
cap = cv2.VideoCapture(0)
while True:
    ret , img = cap.read()
    if not ret:
        continue
    gray = cv2.cvtColor(img , cv2.COLOR_BGR2GRAY)
    face = face_cascade.detectMultiScale(gray , 1.3  ,5)
    for x,y,w,h in face:
        cv2.rectangle(img , (x , y), (x+w , y+h) , (255 , 0, 0), 2)
        croped_face = gray[y:y+h ,x:x+h]
        croped_face = cv2.resize(croped_face , (48 , 48))
        img_pixels = image.img_to_array(croped_face)
        img_pixels = np.expand_dims(img_pixels, axis = 0)
        img_pixels/=255
        
        predictions = model.predict(img_pixels)
#         print(predictions)
        max_index = np.argmax(predictions[0])
        emotions = ('angry','fear', 'happy', 'neutral', 'sad', 'suprise')
#         emotions = ("संतप्त",'भीती', 'आनंदी', 'तटस्थ', 'दु:खी', 'आश्चर्य')
        predicted_emotion = emotions[max_index]

        cv2.putText(img, predicted_emotion, (int(x), int(y)), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255), 2)
     
    resized_img = cv2.resize(img, (1000, 700))
    cv2.imshow('Facial emotion analysis ',resized_img)
    k = cv2.waitKey(30) & 0xff
    if k == 27:
        break
        
cap.release()
cv2.destroyAllWindows()